In [1]:
import numpy as np
import cv2
import pickle
from tensorflow import keras

In [2]:
frameWidth = 7000
frameHeight = 7000
brightness = 300
threshold = 0.90
font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10, brightness)
pickle_in = open("model_trained.p", "rb")
model = pickle.load(pickle_in)

Using TensorFlow backend.


In [3]:
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img
def equalize(img):
    img = cv2.equalizeHist(img)
    return img

def pre_processing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img / 255
    return img

In [4]:
def getClassName(classNo):
    if classNo == 0:
        return 'CORN PLANT LEAF CONDITION: CORN GRAY LEAF SPOT, Reason: Grey leaf spot (GLS) is a foliar fungal disease that affects corn. Symptoms: Symptoms first appear on lower leaves about two to three weeks before tasseling. Disease Management: Crop rotation, Resistant varieties, Residue management,    Weed Control, and Fungicides like Headline EC, Quilt , Proline 480 SC, Tilt 250 E, Bumper 418 EC.'
    elif classNo == 1:
        return 'CORN PLANT LEAF CONDITION: CORN COMMON RUST. Reason: Common rust is caused by the fungus Puccinia sorghi. Symptoms: chlorotic flecks on the leaf surface. Younger leaves are more susceptible to infection, Disease Managment: use of resistant maize hybrids, foliar application of fungicide and cultural control, Early planting time can help avoid P. sorghi. Debris should be collected and destroyed by burning along with eradication of Oxalis in surrounding areas'
    elif classNo == 2:
        return 'CORN PLANT LEAF CONDITION: CORN HEALTHY, Techniques and methods redarding how to yield corn crop more: Plant Early, Plant Effectively, Practice Seasonal Soil Rotation, Know The Yield Potential, Always Scout the Fields, Ensure Proper Water Drainage, Utilize Fertilizers, Test the Soil, Weed Early and Often, Seed Quality'
    elif classNo == 3:
        return 'CORN PLANT LEAF CONDITION: NORTHEN CORN LEAF BLIGHT (NCLB). Reason: It is a foliar disease of corn caused by Exserohilum turcicum. Symptoms: cigar-shaped or elliptical necrotic gray-green lesions on the leaves that range from one to seven inches long. Disease Management: Preventative management strategies, Different cropping practices, fungicides, Scouting fields and monitoring local conditions, managing weeds, 1-2 year rotation of corn.'
    elif classNo == 4:
        return 'POTATO PLANT LEAF CONDITION: POTATO EARLY BLIGHT, Reason: Alternaria solani is a fungal pathogen that produces a disease in potato plants called early blight, Disease Management: Cultural control like: Clearing infected debris, Watering the plants at the right time and certain amount of it, drip irrigation system, controling the wild population of Solanaceae, Monitoring the field. Chemical control: pyraclostrobin, Bacillus subtilis, chlorothalonil, copper products'
    elif classNo == 5:
        return 'POTATO PLANT LEAF CONDITION: POTATO HEALTHY, Techniques and methods redarding how to yield potato crop more: Plant Early, Plant Effectively, Practice Seasonal Soil Rotation, Know The Yield Potential, Always Scout the Fields, Ensure Proper Water Drainage, Utilize Fertilizers, Test the Soil, Weed Early and Often, Seed Quality'
    elif classNo == 6:
        return 'POTATO PLANT LEAF CONDITION: POTATO LATE BLIGHT, Phytophthora infestans is an oomycete or water mold, a microorganism that causes the serious potato known as late blight or potato blight, Disease Management: Reducing inoculum, Environmental conditions, Use of fungicides, Control of tuber blight'


In [5]:

while True:
    success, imgOriginal = cap.read()

    img = np.asarray(imgOriginal)
    img = cv2.resize(img, (32, 32))
    img = pre_processing(img)
    cv2.imshow("preprocessed Image", img)
    img = img.reshape(1, 32, 32, 1)
    cv2.putText(imgOriginal, "CLASS: ", (20, 35), font, 0.50, (0, 0, 255), 2, cv2.LINE_AA)

    predictions = model.predict(img)
    classIndex = model.predict_classes(img)
    probabilityValue = np.amax(predictions)
    if probabilityValue > threshold:
        cv2.putText(imgOriginal, str(classIndex) + " " + str(getClassName(classIndex)), (120, 35), font, 0.50,
                    (0, 0, 255), 2, cv2.LINE_AA)
        cv2.putText(imgOriginal, str(round(probabilityValue * 100, 2)) + "%", (180, 75), font, 0.50, (255, 0, 0), 2,
                    cv2.LINE_AA)
    cv2.imshow("Result", imgOriginal)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


KeyboardInterrupt: 